In [23]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()

In [19]:
df = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\form.csv', dtype=str)  
df.fillna("",inplace=True)
df = df.astype(str) 
df[0:32]

,FTPE,IDTEXT,SortPos,{0},{1},{2},{3},{4},{5}
0,SDESC,,0,28400,SDESC_0,,3,28400 Mission Critica,r_idt__ ct_SDESC
1,MULTICHECKBOX,1B 1,1,28401,1B_1,,3,28401 What mission-es,r_idt_1B_1 ct_MULTICHECKBOX
2,TXT,1B 1.1,2,28402,1B_1_1,,3,"28402 Other, please d",r_idt_1B_1_1 ct_TXT
3,MULTICHECKBOX,1B 2,3,28403,1B_2,,3,28403 Does your HVA I,r_idt_1B_2 ct_MULTICHECKBOX
4,TXT,1B 2.1,4,28404,1B_2_1,,3,"28404 Other, please d",r_idt_1B_2_1 ct_TXT
5,MULTICHECKBOX,1B 3,5,28405,1B_3,,3,28405 What types of s,r_idt_1B_3 ct_MULTICHECKBOX
6,TXT,1B 3.1,6,28406,1B_3_1,,3,"28406 Other, please d",r_idt_1B_3_1 ct_TXT
7,PICK,1B 4,7,28407,1B_4,,3,28407 your ICS enviro,r_idt_1B_4 ct_PICK
8,TXT,1B 4.1,8,28408,1B_4_1,,3,"28408 Other, please d",r_idt_1B_4_1 ct_TXT
9,PICK,1B 5,9,28409,1B_5,,3,28409 Are there any n,r_idt_1B_5 ct_PICK


In [20]:

st=''
for i,r in df.iterrows():
    tmp = f"C:\_som\_src\SHOWROW.aspx"
    r0 = r['{0}']
    with open(tmp, 'r') as f:
        tmp = f.read() 
    tmp = tmp.replace("{0}", str(r["{0}"])) 
    tmp = tmp.replace("{1}", str(r["{1}"])) 
    tmp = tmp.replace("{2}", str(r["{2}"]))  
    tmp = tmp.replace("{4}", str(r["{4}"]))  
    tmp = tmp.replace("{5}", str(r["{5}"]))  
    st=st+tmp
print(st)

' SDESC_0 28400 28400 Mission Critica
Protected _ShowRow_SDESC_0 As String 
Public Property ShowRow_SDESC_0() As String
Get
    Return _ShowRow_SDESC_0
End Get
Set(ByVal value As String)
    _ShowRow_SDESC_0 = value
End Set
End Property

' 1B_1 28401 28401 What mission-es
Protected _ShowRow_1B_1 As String 
Public Property ShowRow_1B_1() As String
Get
    Return _ShowRow_1B_1
End Get
Set(ByVal value As String)
    _ShowRow_1B_1 = value
End Set
End Property

' 1B_1_1 28402 28402 Other, please d
Protected _ShowRow_1B_1_1 As String 
Public Property ShowRow_1B_1_1() As String
Get
    Return _ShowRow_1B_1_1
End Get
Set(ByVal value As String)
    _ShowRow_1B_1_1 = value
End Set
End Property

' 1B_2 28403 28403 Does your HVA I
Protected _ShowRow_1B_2 As String 
Public Property ShowRow_1B_2() As String
Get
    Return _ShowRow_1B_2
End Get
Set(ByVal value As String)
    _ShowRow_1B_2 = value
End Set
End Property

' 1B_2_1 28404 28404 Other, please d
Protected _ShowRow_1B_2_1 As String 
Public Pr

In [26]:
t = """
Has the as Languages technologies platforms is your web don't we have web driver 
"""
def enum_encode(t):
    t = re.sub('[^A-Za-z0-9]', ' ', t ) 
    t = ' '.join([w[:15] for w in t.split(' ') if w not in sw])
    #t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper().strip()
    t = re.sub('\s', '_', t) 
    #t = re.sub('[AEIOU\s]', '', t)  
    return t
enum_encode(t)

'HAS_LANGUAGES_TECHNOLOGIES_PLATFORMS_WEB_WEB_DRIVER'

In [27]:
engine = create_engine(connstr)
conn = engine.connect()
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType IN (
	SELECT DISTINCT PK_PickListType
	FROM vwQuestions WHERE PK_FORM LIKE '2022-A-HVA' AND PK_PickListType IS NOT NULL --SectionNum=1
)
"""  
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"
s = s + f"Enum PICK_{UsageField} '{dfpicks.iloc[0]['Description']}-{dfpicks.iloc[0]['PK_PickListType']}\n"
for i,r in dfpicks.iterrows():
    if UsageField != dfpicks.iloc[i]['UsageField']:
        UsageField=dfpicks.iloc[i]['UsageField']
        s = s + f"End Enum\n"
        #s = s + f"\n'{dfpicks.iloc[i]['Description']}-{dfpicks.iloc[i]['PK_PickListType']}\n"
        s = s + f"Enum PICK_{UsageField} '{dfpicks.iloc[i]['Description']}-{dfpicks.iloc[i]['PK_PickListType']}\n"
    s = s + f"\t{r.CodeValue}_{enum_encode(str(r.DisplayValue)[:15])} = {str(r.PK_PickList)} '{str(r.DisplayValue)[:45]}\n" 
s=s+"End Enum\n"
print(s)
conn.close()


Enum PICK_HVATECH 'HVA System Technology-153
	CLOUD_CLOUD = 3412 'Cloud
	MAIN_MAINFRAME = 3413 'Mainframe
	GSS_GSS = 3414 'GSS
	WEB_WEB_APP = 3415 'Web App
	APP_APPLICATION_PLA = 3416 'Application Platform
	ISC_ICS_SCADA = 3417 'ICS/SCADA
	CS_CLIENT_SERVER = 3418 'Client/Server
	OTH_OTHER = 3419 'Other
End Enum
Enum PICK_CLOUD 'Cloud Types-154
	PAAS_PAAS = 3424 'Paas
	SAAS_SAAS = 3425 'Saas
	IAAS_IAAS = 3426 'Iaas
	HYBRID_HYBRID = 3427 'Hybrid
	OTH_OTHER = 3428 'Other
End Enum
Enum PICK_HVAMGMT 'HVA System Management Type-155
	GOGO_GOVERNMENT_OWNE = 3432 'Government owned, Government operated (GoGo)
	GOCO_GOVERNMENT_OWNE = 3433 'Government owned, Contractor operated (GoCo)
	COCO_CONTRACTOR_OWNE = 3434 'Contractor owned, Contractor operated (CoCo)
	FFRDC_FEDERALLY_FUNDE = 3435 'Federally Funded Research and Development Cen
	OTH_OTHER = 3436 'Other
End Enum
Enum PICK_HVAMAINT 'HVA Maintenance-156
	FED_FEDERAL_EMPLOYE = 3441 'Federal employees
	FEDCONT_FEDERAL_EMPLOYE = 3442 'Federal emp